# loading Bronze data

In [9]:
customers_raw=spark.read.parquet("abfss://e0cf1801-1533-430a-8805-b23abd9dafae@onelake.dfs.fabric.microsoft.com/1a49c4d3-a3e2-4ff9-a637-24c27f990123/Files/BRONZE/customers.parquet")
orders_raw=spark.read.parquet("abfss://e0cf1801-1533-430a-8805-b23abd9dafae@onelake.dfs.fabric.microsoft.com/1a49c4d3-a3e2-4ff9-a637-24c27f990123/Files/BRONZE/orders.parquet")
payments_raw=spark.read.parquet("abfss://e0cf1801-1533-430a-8805-b23abd9dafae@onelake.dfs.fabric.microsoft.com/1a49c4d3-a3e2-4ff9-a637-24c27f990123/Files/BRONZE/payments.parquet")
support_raw=spark.read.parquet("abfss://e0cf1801-1533-430a-8805-b23abd9dafae@onelake.dfs.fabric.microsoft.com/1a49c4d3-a3e2-4ff9-a637-24c27f990123/Files/BRONZE/support_tickets.parquet")
webactivity_raw=spark.read.parquet("abfss://e0cf1801-1533-430a-8805-b23abd9dafae@onelake.dfs.fabric.microsoft.com/1a49c4d3-a3e2-4ff9-a637-24c27f990123/Files/BRONZE/web_activities.parquet")


StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 11, Finished, Available, Finished)

In [10]:
display(customers_raw.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7d4d9515-89f6-46ac-adaf-6d5a0582be59)

In [11]:
display(orders_raw.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 866c6a95-55c0-4306-8180-bb3d5ba61ee5)

In [12]:
display(payments_raw.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ef92972a-9a4f-4497-b66c-29787029fbfd)

In [13]:
display(support_raw.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cbdeb815-2683-4be4-93c7-5b018977e5c9)

In [14]:
display(webactivity_raw.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b362dcb9-173a-4c93-ae47-83201c8f70aa)

In [16]:
customers_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_customer")
orders_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_orders")
payments_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_payments")
support_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_support")
webactivity_raw.write.format("delta").mode("overwrite").saveAsTable("bronze_webactivity")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 18, Finished, Available, Finished)

## data cleaning for silver 

In [18]:
%%sql
select * from bronze_customer limit 5

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 20, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 6 fields>

In [20]:
customer_raw=spark.table("bronze_customer")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 22, Finished, Available, Finished)

In [24]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 26, Finished, Available, Finished)

In [25]:
customers_clean = (
    customer_raw
    .withColumn("Name", initcap(trim(col("name"))))
    .withColumn("email", lower(trim(col("EMAIL"))))
    .withColumn(
        "Gender",
        when(lower(col("gender")).isin("f", "female"), "Female")
        .when(lower(col("gender")).isin("m", "male"), "Male")
        .otherwise("Neutral")
    )
    .withColumn(
        "DOB",
        to_date(regexp_replace(col("dob"), "/", "-"))
    )
    .withColumn("Location", initcap(lower(col("location"))))
    .dropDuplicates(subset=["customer_id"])
    .dropna(subset=["customer_id", "email"])
)

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 27, Finished, Available, Finished)

In [26]:
display(customers_clean.limit(6))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f69e6f92-b70b-4b05-9e8d-8e69809ce11b)

In [27]:
%%sql
select * from bronze_orders limit 5

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 29, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 5 fields>

In [29]:
orders_raw= spark.table("bronze_orders")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 31, Finished, Available, Finished)

In [32]:
orders_clean = (
    orders_raw
    # Clean Order_date with multiple patterns
    .withColumn(
        "Order_date",
        when(
            col("order_date").rlike(r"^\d{4}/\d{2}/\d{2}$"),
            to_date(col("order_date"), "yyyy/MM/dd")
        )
        .when(
            col("order_date").rlike(r"^\d{2}-\d{2}-\d{4}$"),
            to_date(col("order_date"), "dd-MM-yyyy")
        )
        .when(
            col("order_date").rlike(r"^\d{8}$"),
            to_date(col("order_date"), "yyyyMMdd")
        )
        .otherwise(to_date(col("order_date"), "yyyy-MM-dd"))
    )

    # Cast Amount
    .withColumn("Amount", col("amount").cast(DoubleType()))

    # Fix invalid Amount: replace negative/zero with null
    .withColumn(
        "Amount",
        when(col("Amount") < 0, None).otherwise(col("Amount"))
    )

    # Clean Status column
    .withColumn("Status", initcap(lower(col("status"))))

    # Remove duplicate Orders
    .dropDuplicates(subset=["customer_id", "order_id"])

    # Drop rows with missing order_id
    .dropna(subset=["order_id"])
)

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 34, Finished, Available, Finished)

In [33]:
display(orders_clean.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 35, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ea010297-f82e-46c0-99b7-3db22e96593a)

In [34]:
%%sql
select * from bronze_payments limit 5

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 36, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 6 fields>

In [36]:
payment_raw= spark.table("bronze_payments")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 38, Finished, Available, Finished)

In [39]:
payments_clean = (
    payment_raw
    # Convert payment_date → replace "/" with "-" → to_date()
    .withColumn(
        "payment_date",
        to_date(regexp_replace(col("payment_date"), "/", "-"))
    )

    # Clean payment method
    .withColumn("payment_method", initcap(col("payment_method")))
    .replace({"Creditcard": "Credit Card"}, subset=["payment_method"])

    # Clean payment status
    .withColumn("payment_status", initcap(col("payment_status")))

    # Cast Amount to double
    .withColumn("Amount", col("amount").cast(DoubleType()))

    # Fix negative amount → replace with null
    .withColumn(
        "Amount",
        when(col("Amount") < 0, None).otherwise(col("Amount"))
    )

    # Drop rows with missing required fields
    .dropna(subset=["customer_id", "payment_date", "Amount"])
)

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 41, Finished, Available, Finished)

In [43]:
display(payments_clean.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 45, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b30a472-524c-4b8a-94b7-7e235b66e594)

In [41]:
%%sql
select * from bronze_support limit 5

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 43, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 5 fields>

In [44]:
support_raw= spark.table("bronze_support")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 46, Finished, Available, Finished)

In [68]:
support_clean = (
    support_raw
    .withColumn("issue_type", initcap(trim(col("issue_type"))))
    .withColumn("resolution_status", initcap(trim(col("resolution_status"))))

    # Clean ticket_date with multiple patterns
    .withColumn(
        "ticket_date",   
         to_date(regexp_replace(col("ticket_date"),"/","-"))  # fallback
    )

    # Replace NA/blank
    .replace({"NA": None, "": None}, subset=["issue_type", "resolution_status"])

    # Deduplicate
    .dropDuplicates(subset=["ticket_id"])

    # Remove rows with missing keys
    .dropna(subset=["customer_id", "ticket_date","resolution_status"])
)


StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 70, Finished, Available, Finished)

In [69]:
display(support_clean.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 71, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d978fb92-9498-4da8-8bdb-ed1fec2c457f)

In [61]:
%%sql 
select * from bronze_webactivity limit 5

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 63, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 5 fields>

In [63]:
web_raw = spark.table("bronze_webactivity")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 65, Finished, Available, Finished)

In [64]:
web_clean = (
    web_raw
    .withColumn("session_time", to_date(regexp_replace(col("session_time"), "/", "-")))
    .withColumn("page_viewed", lower(col("page_viewed")))
    .withColumn("device_type", initcap(col("device_type")))
    .dropDuplicates(["session_id"])
    .dropna(subset=["customer_id", "session_time", "page_viewed"])
)

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 66, Finished, Available, Finished)

In [65]:
display(web_clean.limit(5))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 67, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 37123676-c033-4dce-9a3e-1ef853c88dde)

In [73]:
customers_clean.write.format("delta").mode("overwrite").saveAsTable("silver_customer")
orders_clean.write.format("delta").mode("overwrite").saveAsTable("silver_order")
payments_clean.write.format("delta").mode("overwrite").saveAsTable("silver_payments")
support_clean.write.format("delta").mode("overwrite").saveAsTable("silver_support")
web_clean.write.format("delta").mode("overwrite").saveAsTable("silver_web_activity")


StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 75, Finished, Available, Finished)

### **GOLD LAYER***

In [75]:
cust=spark.table("silver_customer").alias("c")
ord=spark.table("silver_order").alias("o")
pay=spark.table("silver_payments").alias("p")
sup=spark.table("silver_support").alias("s")
web=spark.table("silver_web_activity").alias("w")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 77, Finished, Available, Finished)

In [76]:
customerGOLD = (
    cust
    .join(ord, "customer_id", "left")
    .join(pay, "customer_id", "left")
    .join(sup, "customer_id", "left")
    .join(web, "customer_id", "left")
    .select(
        col("c.customer_id"),
        col("c.Name"),
        col("c.email"),
        col("c.Gender"),
        col("c.DOB"),
        col("c.Location"),

        col("o.order_id"),
        col("o.Order_date"),
        col("o.Amount").alias("order_amount"),
        col("o.Status").alias("order_status"),

        col("p.payment_method"),
        col("p.payment_status"),
        col("p.Amount").alias("payment_amount"),

        col("s.ticket_id"),
        col("s.issue_type"),
        col("s.ticket_date"),
        col("s.resolution_status"),

        col("w.page_viewed"),
        col("w.device_type"),
        col("w.session_time")
    )
)

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 78, Finished, Available, Finished)

In [77]:
display(customerGOLD.limit(10))

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 79, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e0aee94d-7721-472f-8dd1-71747db23ea5)

In [79]:
customerGOLD.write.format("delta").mode("overwrite").saveAsTable("GOLD_customers")

StatementMeta(, 1f5091fb-a05d-4cf8-bdce-530df623f548, 81, Finished, Available, Finished)